In [1]:
#
# This notebook will be used to load the Midas dataset and produce a dictionary of parameters to be used for modeling
#

In [2]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? n
Nothing done.


In [3]:
ForColab = False

In [4]:
if ForColab:
    !pip install wget
    import plotly.io as pio
    pio.renderers.default = 'colab'
else:
    print("Working local")

Working local


In [5]:
#
# Based on the SEIR-HCD model code from kaggle, I have added the online data for the world,
# one can choose which country they want, model to use and the date to use as initial start day
#

In [6]:
import matplotlib.pyplot as plt
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (12, 10),
         'axes.labelsize': 13,
         'axes.titlesize':13,
         'xtick.labelsize':13,
         'ytick.labelsize':13,
         'axes.labelweight':'bold',
          'legend.fontsize': 13,
         'figure.max_open_warning': 0}
         
plt.rcParams.update(params)

In [7]:
from datetime import datetime, timedelta
import os
from pathlib import Path

import wget
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
    
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm

from scipy.integrate import odeint, solve_ivp
from scipy.optimize import minimize,curve_fit

from sklearn.metrics import mean_squared_log_error, mean_squared_error

from PIL import Image

import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)



In [8]:
def get_params(ParamEstimatesDf, ColName, ParamName, SA):
    '''
    
    This function will search the midas dataset for parameters
    usage: get_params(ParamEstimatesDf, ColName, ParamName, SA)
    Input:
        Dataframe name, column name, parameter name and SA which is 'y/n'
        If SA == 'y' the function will look only for South Africa data
        else if will list the ountries for which the data are available
    Output:
        return a dataframe that contains the list of paarametrs with some pre-seclected columns
    
    '''
    ValidName = ParamEstimatesDf[ColName].unique()
    ColToList = ['peer_review','name', 'abbreviation',
   'units', 'country', 'start_date',
   'end_date', 'value_type', 'value', 'uncertainty_type', 'lower_bound',
   'upper_bound', 'population', 'age_range', 'method_description',
   'data_description', 'title_publication',
   'type_publication', 'authors', 'publication_URL']
    
    try:
        WhatToSearch = ParamEstimatesDf[(ParamEstimatesDf[ColName] == ParamName)]
    except:
        print(f"Something went wrong, enter any of the following {ValidName}")

    WhatToSearch['value'].replace({"Unspecified":np.nan},inplace=True)
    WhatToSearch['value'].dropna(inplace=True)

    if len(WhatToSearch['country'].unique())<5:
        print(f"Available countries are {WhatToSearch['country'].unique()}")
        tmp = WhatToSearch[WhatToSearch['country'] == input("Enter country ")]#['name','abbreviation','units','country']
    elif SA.lower() =='y' and 'South Africa' in (WhatToSearch['country'].unique()):
        print("South Africa value available ")
        tmp = WhatToSearch[WhatToSearch['country'] == 'South Africa']
    else:
        tmp = WhatToSearch[WhatToSearch['country'] == input("Enter country ")]
        
    print("\n")
    PeerReviewed = tmp[(tmp['peer_review'] == 'positive')][ColToList]

    if len(PeerReviewed) >= 1:
        print(f"\033[32m Peer reviewed values are available \033[0m")
        for (idx, row) in PeerReviewed.iterrows():
            for j in range(len(ColToList)):
                print(f"\033[1m{ColToList[j]}:\033[0m {row.loc[ColToList[j]]}")
            print("\n")
        return(PeerReviewed)
    else:
        print(f"\033[31m Non per reviewed values will be used \033[0m")
        #for i in range(tmp.shape[0]):
        #    print(f"{tmp.iloc[:,:].values[i]}")
        for (idx, row) in tmp.iterrows():
            for j in range(len(ColToList)):
                print(f"\033[1m{ColToList[j]}:\033[0m {row.loc[ColToList[j]]}")
            print("\n")
        return(tmp[ColToList])



Parameters used in the model
R_t = reproduction number at time t. Typical 3.6* at t=0

Transition times

- T_inc = average incubation period. Typical 5.6* days
- T_inf = average infectious period. Typical 2.9 days
- T_hosp = average time a patient is in hospital before either recovering or becoming critical. Typical 4 days
- T_crit = average time a patient is in a critical state (either recover or die). Typical 14 days

Fractions These constants are likely to be age specific (hence the subscript a):

- m_a = fraction of infections that are asymptomatic or mild. Assumed 80% (i.e. 20% severe)
- c_a = fraction of severe cases that turn critical. Assumed 10%
- f_a = fraction of critical cases that are fatal. Assumed 30%
- *Averages taken from https://www.kaggle.com/covid-19-contributions

    :param t: Time step for solve_ivp
    :param y: Previous solution or initial values
    :param R_t: Reproduction number
    :param t_inc: Average incubation period. Default 5.2 days
    :param t_inf: Average infectious period. Default 2.9 days
    :param t_hosp: Average time a patient is in hospital before either recovering or becoming critical. Default 4 days
    :param t_crit: Average time a patient is in a critical state (either recover or die). Default 14 days
    :param m_a: Fraction of infections that are asymptomatic or mild. Default 0.8
    :param c_a: Fraction of severe cases that turn critical. Default 0.1
    :param f_a: Fraction of critical cases that are fatal. Default 0.3
    :return:

## SA Hospital data

In [9]:
def get_sa_hospital(ToUpdateSAData):
    if ToUpdateSAData.lower()=='y':
        url= "https://raw.githubusercontent.com/dsfsi/covid19za/Hospital_Data/data/health_system_za_public_hospitals_extended_details.csv"
        HospitalDf = pd.read_csv(wget.download(url))
    else:
        HospitalDf = pd.read_csv('health_system_za_public_hospitals_extended_details.csv')
    HospitalWithICU = HospitalDf.loc[np.where(HospitalDf.speciality_services.str.contains('intensive care') == True)]

    print(f"Mean Capacity \n{HospitalWithICU[['minimum_capacity','maximum_capacity']].mean()} \n Total Capacity \n{HospitalWithICU[['minimum_capacity','maximum_capacity']].sum()}")
    
    return(HospitalWithICU[['minimum_capacity','maximum_capacity']].mean(),
          HospitalWithICU[['minimum_capacity','maximum_capacity']].sum())

In [10]:
SAHospital = get_sa_hospital('y')

Mean Capacity 
minimum_capacity    371.428571
maximum_capacity    771.428571
dtype: float64 
 Total Capacity 
minimum_capacity     7800.0
maximum_capacity    16200.0
dtype: float64


In [11]:
MeanTotalBed = np.mean([SAHospital[1][0],SAHospital[1][1]])

# Parameters estimates for other models

## Location of various parameters as per Midas 

### https://github.com/midas-network/COVID-19/blob/master/parameter_estimates/2019_novel_coronavirus/estimates.csv
### Check above link for updated parameters

Paramters used:

- Average incubation period, t_inc
- Average infection period, t_inf
- Average hospitalization period, t_hosp
- Average critital period, t_crit
- The fraction of mild/asymptomatic cases, m_a
- The fraction of severe cases that turn critical, c_a
- The fraction of critical cases that result in a fatality, f_a
- Reproduction number, R_0 or R_t

In [12]:
#
# This is not working properly 
#

def estimates(ToUpdate):

    url = "https://raw.githubusercontent.com/midas-network/COVID-19/master/parameter_estimates/2019_novel_coronavirus/estimates.csv"
    if ToUpdate.lower()=='y':
        ParamEstimates = pd.read_csv(wget.download(url))
    else:
        ParamEstimates = pd.read_csv("estimates.csv")

    #ParamEstimatesDf['peer_review'].unique()
    ParamEstimates['peer_review']  = ParamEstimates['peer_review'].replace({'postitive':'positive'})
    ParamEstimates['peer_review']  = ParamEstimates['peer_review'].replace({np.nan:'Not_Available'})
    
    return(ParamEstimates)

In [13]:
ParamEstimatesDf = estimates('y')

In [14]:
ParamEstimatesDf.columns

Index(['id', 'peer_review', 'peer_review_location', 'name', 'abbreviation',
       'units', 'country', 'location_name', 'location_type', 'start_date',
       'end_date', 'value_type', 'value', 'uncertainty_type', 'lower_bound',
       'upper_bound', 'population', 'age_range', 'method_description',
       'data_description', 'data_URL', 'date_publication', 'title_publication',
       'type_publication', 'authors', 'publication_URL', 'entry_date',
       'entry_person', 'entry_email'],
      dtype='object')

In [15]:
ValidAbreviation = ParamEstimatesDf['abbreviation'].unique()

In [16]:
ValidAbreviation

array([nan, 'R0', 'k', 'CFR', 'RE', 'gamma', 'r', 'R', 'Rt', 'K', 'Rv',
       'Z', 'beta', 'alpha', 'IFR', 'Reff', 'X0', 'Xf', 'r0', 'rf'],
      dtype=object)

In [17]:
#
# R, RE, Reff, Rt, rt, Rv are all the same thing - replace them as Re
#

In [18]:
ParamEstimatesDf['name'] = ParamEstimatesDf['name'].str.casefold()

In [19]:
#
# 'basic reprodutive number' and 'basic reprodution number' are same
#
ParamEstimatesDf['name'][ParamEstimatesDf.name.str.contains('basic reprodutive number')] = 'basic reproduction number'
ParamEstimatesDf['name'][ParamEstimatesDf.name.str.contains('basic reprodution number')] = 'basic reproduction number'
ParamEstimatesDf['name'][ParamEstimatesDf.name.str.contains('basic reproductive number')] = 'basic reproduction number'


In [20]:
ParamEstimatesDf['abbreviation'][ParamEstimatesDf.name.str.contains('effective reproduction number')] = 'Re'

In [21]:
# r0, r and rf are the same 
#ParamEstimatesDf[ParamEstimatesDf['abbreviation'] == 'r']['name']
ParamEstimatesDf['abbreviation'][ParamEstimatesDf.name.str.contains('growth rate')] = 'r'


In [22]:
ParamEstimatesDf['abbreviation'].unique()

array([nan, 'R0', 'k', 'CFR', 'Re', 'r', 'gamma', 'K', 'Z', 'beta',
       'alpha', 'IFR', 'X0', 'Xf'], dtype=object)

In [23]:
ColToList = ['peer_review','name', 'abbreviation',
   'units', 'country', 'start_date',
   'end_date', 'value_type', 'value', 'uncertainty_type', 'lower_bound',
   'upper_bound', 'population', 'age_range', 'method_description',
   'data_description', 'title_publication',
   'type_publication', 'authors', 'publication_URL']

In [24]:
ValidName = ParamEstimatesDf['name'].unique()

In [25]:
#ParamEstimatesDf['name'].unique()
#ParamEstimatesDf[ParamEstimatesDf['name'] == 'basic reprodution number']

In [26]:
ParamEstimatesDf.to_excel("estimates.xls",index=False)
ParamEstimatesDf.to_csv("estimates.csv",index=False)
!pwd

/Users/nadeem/GitHub/COVID-19


In [27]:
# Example if one wants to get only one parameter

## Getting R0

In [28]:
df = get_params(ParamEstimatesDf,'name','basic reproduction number','y')

South Africa value available 


 Non per reviewed values will be used 
peer_review: Not_Available
name: basic reproduction number
abbreviation: R0
units: Nb
country: South Africa
start_date: Unspecified
end_date: 2020-05-10
value_type: Unspecified
value: 2.7
uncertainty_type: Unspecified
lower_bound: Unspecified
upper_bound: Unspecified
population: general population
age_range: Unspecified
method_description: developed an SEIR model with two main assumptions to make it generalizable to more countries: that the existing influences of policies or culture of the target object will be unchanged, and that during the transmission of the virus, the probability of infection in contact with each person is equal in the same target group
data_description: data was from Johns Hopkins University, for 222 countries
title_publication: Assessing the Global Tendency of COVID-19 Outbreak
type_publication: preprint
authors: Liu Q., Liu Z., Zhu J., Zhu Y., Li D., Gao Z., Zhou L., Tang Y., Zhang X., Yang J

In [29]:
df

,peer_review,name,abbreviation,units,country,start_date,end_date,value_type,value,uncertainty_type,lower_bound,upper_bound,population,age_range,method_description,data_description,title_publication,type_publication,authors,publication_URL
356,Not_Available,basic reproduction number,R0,Nb,South Africa,Unspecified,2020-05-10,Unspecified,2.7,Unspecified,Unspecified,Unspecified,general population,Unspecified,developed an SEIR model with two main assumpti...,"data was from Johns Hopkins University, for 22...",Assessing the Global Tendency of COVID-19 Outb...,preprint,"Liu Q., Liu Z., Zhu J., Zhu Y., Li D., Gao Z.,...",https://www.medrxiv.org/content/10.1101/2020.0...


In [30]:
R0_SA = float(df['value'].values[0])

In [31]:
R0_SA

2.7

In [33]:
dfChina = get_params(ParamEstimatesDf,'name', 'basic reproduction number','n')

Enter country China


 Peer reviewed values are available 
peer_review: positive
name: basic reproduction number
abbreviation: R0
units: Nb
country: China
start_date: 2020-01-09
end_date: 2020-01-11
value_type: Unspecified
value: 0.3
uncertainty_type: confidence interval 95%
lower_bound: 0.17
upper_bound: 0.44
population: general population
age_range: Unspecified
method_description: Using the method described by Cauchemez, assuming that the market was the only source of zoonotic infections
data_description: first 41 confirmed cases
title_publication: Real-time tentative assessment of the epidemiological characteristics of novel coronavirus infections in Wuhan, China, as at 22 January 2020
type_publication: report
authors: Wu P., Hao X., Lau EHY., Wong JY., Leung KSM., Wu JT., Cowling BJ., Leung GB.  
publication_URL: https://eurosurveillance.org/content/10.2807/1560-7917.ES.2020.25.3.2000044#r16


peer_review: positive
name: basic reproduction number
abbreviation: R0
units: Nb
country:

In [34]:
tmp = dfChina[dfChina['peer_review'] == 'positive']

In [35]:
ToPrint = tmp[(tmp['value_type'] != 'Unspecified') & (tmp['peer_review'] == 'positive')][ColToList]

In [36]:
ToPrint

,peer_review,name,abbreviation,units,country,start_date,end_date,value_type,value,uncertainty_type,lower_bound,upper_bound,population,age_range,method_description,data_description,title_publication,type_publication,authors,publication_URL
6,positive,basic reproduction number,R0,Nb,China,2020-01-10,2020-01-24,mean,2.24,confidence interval 95%,1.96,2.55,general population,Unspecified,modeled the epidemic curve of 2019-nCoV cases ...,used case time series data,Preliminary estimation of the basic reproducti...,preprint,"Zhao S., Ran J., Musa SS., Yang G., Lou Y., Ga...",https://www.biorxiv.org/content/10.1101/2020.0...
473,positive,basic reproduction number,R0,Nb,China,2020-01-15,2020-01-20,median,5.7,confidence interval 95%,3.8,8.9,NaN,Unspecified,Considered realistic distributions for latent ...,Used publicly available reports of 140 confirm...,High Contagiousness and Rapid Spread of Severe...,journal article,"Sanche S., Lin TY., Xu C., Romero-Severson E.,...",https://wwwnc.cdc.gov/eid/article/26/7/20-0282...


In [108]:
Rtdf = get_params(ParamEstimatesDf,'name','effective reproduction number','y')

Enter country China


 Peer reviewed values are available 
peer_review: positive
name: effective reproduction number
abbreviation: Re
units: Nb
country: China
start_date: 2020-01-10
end_date: 2020-01-23
value_type: median
value: 2.38
uncertainty_type: confidence interval 95%
lower_bound: 2.04
upper_bound: 2.77
population: general population
age_range: Unspecified
method_description: developed a mathematical model that simulates the spatio-temporal 
dynamics of infections among 375 Chinese cities
data_description: Daily numbers of travelers between 375 Chinese cities during the Spring Festival
period (Chunyun) were derived from human mobility data collected by the Tencent 
Location-based Service (LBS) during the 2018 Chunyun period (February 1 to March 12, 2018)
title_publication: Substantial undocumented infection facilitates the rapid dissemination of novel coronavirus (COVID-19)
type_publication: journal article
authors: Li R., Pei S., Chen B., Song Y., Zhang T., Yang W., Shaman J.
p

In [109]:
Rtdf

,peer_review,name,abbreviation,units,country,start_date,end_date,value_type,value,uncertainty_type,lower_bound,upper_bound,population,age_range,method_description,data_description,title_publication,type_publication,authors,publication_URL
176,positive,effective reproduction number,Re,Nb,China,2020-01-10,2020-01-23,median,2.38,confidence interval 95%,2.04,2.77,general population,Unspecified,developed a mathematical model that simulates ...,Daily numbers of travelers between 375 Chinese...,Substantial undocumented infection facilitates...,journal article,"Li R., Pei S., Chen B., Song Y., Zhang T., Yan...",https://science.sciencemag.org/content/early/2...
181,positive,effective reproduction number,Re,Nb,China,2020-01-24,2020-02-03,median,1.36,confidence interval 95%,1.14,1.63,general population,Unspecified,re-estimated system parameters using metapopul...,Daily numbers of travelers between 375 Chinese...,Substantial undocumented infection facilitates...,journal article,"Li R., Pei S., Chen B., Song Y., Zhang T., Yan...",https://science.sciencemag.org/content/early/2...
185,positive,effective reproduction number,Re,Nb,China,2020-01-24,2020-02-08,median,0.99,confidence interval 95%,0.76,1.33,general population,Unspecified,re-estimated system parameters using metapopul...,Daily numbers of travelers between 375 Chinese...,Substantial undocumented infection facilitates...,journal article,"Li R., Pei S., Chen B., Song Y., Zhang T., Yan...",https://science.sciencemag.org/content/early/2...
519,positive,effective reproduction number,Re,Nb,China,2020-01-21,2020-02-12,Unspecified,3.28,Unspecified,Unspecified,Unspecified,general population,Unspecified,Developed an SIR-X model where X accounts for ...,daily aggregated case numbers provided in same...,Effective containment explains subexponential ...,journal article,"Maier BF., Brockmann D.",https://science.sciencemag.org/content/early/2...
520,positive,effective reproduction number,Re,Nb,China,2020-01-21,2020-02-12,Unspecified,1.93,Unspecified,Unspecified,Unspecified,general population,Unspecified,Developed an SIR-X model where X accounts for ...,daily aggregated case numbers provided in same...,Effective containment explains subexponential ...,journal article,"Maier BF., Brockmann D.",https://science.sciencemag.org/content/early/2...
521,positive,effective reproduction number,Re,Nb,China,2020-01-21,2020-02-12,Unspecified,3.02,Unspecified,Unspecified,Unspecified,general population,Unspecified,Developed an SIR-X model where X accounts for ...,daily aggregated case numbers provided in same...,Effective containment explains subexponential ...,journal article,"Maier BF., Brockmann D.",https://science.sciencemag.org/content/early/2...
522,positive,effective reproduction number,Re,Nb,China,2020-01-21,2020-02-12,Unspecified,2.41,Unspecified,Unspecified,Unspecified,general population,Unspecified,Developed an SIR-X model where X accounts for ...,daily aggregated case numbers provided in same...,Effective containment explains subexponential ...,journal article,"Maier BF., Brockmann D.",https://science.sciencemag.org/content/early/2...
523,positive,effective reproduction number,Re,Nb,China,2020-01-21,2020-02-12,Unspecified,3.1,Unspecified,Unspecified,Unspecified,general population,Unspecified,Developed an SIR-X model where X accounts for ...,daily aggregated case numbers provided in same...,Effective containment explains subexponential ...,journal article,"Maier BF., Brockmann D.",https://science.sciencemag.org/content/early/2...
524,positive,effective reproduction number,Re,Nb,China,2020-01-21,2020-02-12,Unspecified,3.28,Unspecified,Unspecified,Unspecified,general population,Unspecified,Developed an SIR-X model where X accounts for ...,daily aggregated case numbers provided in same...,Effective containment explains subexponential ...,journal article,"Maier BF., Brockmann D.",https://science.sciencemag.org/content/early/2...
525,positive,effective reproduction number,Re,Nb,China,2020-01-21,2020-02-12,U

# SympDead

In [37]:
SympDead = get_params(ParamEstimatesDf,'name','proportion of symptomatics that die','y') 

Available countries are ['China']
Enter country China


 Non per reviewed values will be used 
peer_review: Not_Available
name: proportion of symptomatics that die
abbreviation: CFR
units: proportion
country: China
start_date: 2019-12-29
end_date: 2020-01-23
value_type: Unspecified
value: 0.03
uncertainty_type: Unspecified
lower_bound: Unspecified
upper_bound: Unspecified
population: general population
age_range: Unspecified
method_description: nan
data_description: 830 confirmed cases in China up to Jan 23
title_publication: Transmission dynamics of 2019 novel coronavirus (2019-nCoV)
type_publication: preprint
authors: Liu T., Hu J., Kang M., Lin L., Zhong H., Xiao J., He G., Song T., Huang Q., Rong Z., Deng A., Zeng W., Tan X., Zeng S., Zhu Z., Li J., Wan D., Lu J., Deng H., He J., Ma W.
publication_URL: https://www.biorxiv.org/content/10.1101/2020.01.25.919787v1


peer_review: Not_Available
name: proportion of symptomatics that die
abbreviation: CFR
units: proportion
country: China


In [38]:
SympDead

,peer_review,name,abbreviation,units,country,start_date,end_date,value_type,value,uncertainty_type,lower_bound,upper_bound,population,age_range,method_description,data_description,title_publication,type_publication,authors,publication_URL
11,Not_Available,proportion of symptomatics that die,CFR,proportion,China,2019-12-29,2020-01-23,Unspecified,0.03,Unspecified,Unspecified,Unspecified,general population,Unspecified,NaN,830 confirmed cases in China up to Jan 23,Transmission dynamics of 2019 novel coronaviru...,preprint,"Liu T., Hu J., Kang M., Lin L., Zhong H., Xiao...",https://www.biorxiv.org/content/10.1101/2020.0...
13,Not_Available,proportion of symptomatics that die,CFR,proportion,China,2019-12-31,2020-02-05,Unspecified,0.18,credible interval 95%,0.11,0.81,general population,Unspecified,parametric model assuming exponential growth,publicly reported cases and deaths,Report 4:Severity of 2019-novel coronavirus (n...,report,"Dorigatt I., Okell L., Cori A., Imai N., Bague...",https://www.imperial.ac.uk/mrc-global-infectio...
46,Not_Available,proportion of symptomatics that die,CFR,proportion,China,2019-12-08,2020-01-24,Unspecified,5.30%,confidence interval 95%,3.50%,7.50%,general population,Unspecified,estimate growth rate from 8 december,use 23 exported cases,Real time estimation of the risk of death from...,preprint,"Jung S., Akhmetzhanov AR., Hayashi A., Linton ...",https://www.medrxiv.org/content/10.1101/2020.0...
49,Not_Available,proportion of symptomatics that die,CFR,proportion,China,2019-12-08,2020-01-24,Unspecified,8.40%,confidence interval 95%,5.30%,12.30%,general population,Unspecified,estimate growth rate from 8 december,"use data since growth of exported cases, 20 ex...",Real time estimation of the risk of death from...,preprint,"Jung S., Akhmetzhanov AR., Hayashi A., Linton ...",https://www.medrxiv.org/content/10.1101/2020.0...
59,Not_Available,proportion of symptomatics that die,CFR,proportion,China,2019-12-31,2020-02-13,Unspecified,0.0406,Unspecified,Unspecified,Unspecified,general population,Unspecified,reconstructed underlying transmission dynamics,daily series of laboratory-confirmed COVID cas...,Early epidemiological assessment of the transm...,preprint,"Mizumoto K., Kagaya K., Chowell G.",https://www.medrxiv.org/content/10.1101/2020.0...


## t_hosp_death - time from hospitalization to death

In [40]:
THospDead = get_params(ParamEstimatesDf,'name','time from hospitalization to death','y') 

Available countries are ['China']
Enter country China


 Peer reviewed values are available 
peer_review: positive
name: time from hospitalization to death
abbreviation: nan
units: days
country: China
start_date: 2020-01-15
end_date: 2020-01-20
value_type: Unspecified
value: 11.2
uncertainty_type: confidence interval 95%
lower_bound: 8.7
upper_bound: 14.9
population: 137 case reports
age_range: Unspecified
method_description: Used hybrid deterministic-stochastic SEIR model and fitted model to new daily case count data reported outside Hubei province during period before substaintial transmission occurred outside of province
data_description: Used publicly available reports of 140 confirmed cases from 15 Jan to 20 Jan from China CDC & Baidu migration server to estimate number of daily travelers in and out of Wuhan
title_publication: High Contagiousness and Rapid Spread of Severe Acute Respiratory Syndrome Coronavirus 2
type_publication: journal article
authors: Sanche S., Lin TY., Xu C.

In [41]:
THospDead

,peer_review,name,abbreviation,units,country,start_date,end_date,value_type,value,uncertainty_type,lower_bound,upper_bound,population,age_range,method_description,data_description,title_publication,type_publication,authors,publication_URL
479,positive,time from hospitalization to death,NaN,days,China,2020-01-15,2020-01-20,Unspecified,11.2,confidence interval 95%,8.7,14.9,137 case reports,Unspecified,Used hybrid deterministic-stochastic SEIR mode...,Used publicly available reports of 140 confirm...,High Contagiousness and Rapid Spread of Severe...,journal article,"Sanche S., Lin TY., Xu C., Romero-Severson E.,...",https://wwwnc.cdc.gov/eid/article/26/7/20-0282...


## Gamma

In [42]:
gammaDF = get_params(ParamEstimatesDf,'abbreviation','gamma','y') 

Available countries are ['China']
Enter country China


 Non per reviewed values will be used 
peer_review: Not_Available
name: symptomatic case fatality ratio
abbreviation: gamma
units: percent
country: China
start_date: 2020-01-11
end_date: 2020-02-10
value_type: mean
value: 2.94
uncertainty_type: confidence interval 90%
lower_bound: 2.9
upper_bound: 3
population: general population
age_range: Unspecified
method_description: estimation of mortality and recovery rates based on formla used by NHC: ratio of cumulative number of recovered/deaths and that of infected at time t
data_description: data publicly available from Hubei from 11 Jan to 10 Feb
title_publication: Data-Based Analysis, Modelling and Forecasting of the COVID-19 outbreak
type_publication: preprint
authors: Anastassopoulou C., Russo L., Tsakris A., Siettos C.
publication_URL: https://www.medrxiv.org/content/10.1101/2020.02.11.20022186v3




In [43]:
ValidAbreviation

array([nan, 'R0', 'k', 'CFR', 'RE', 'gamma', 'r', 'R', 'Rt', 'K', 'Rv',
       'Z', 'beta', 'alpha', 'IFR', 'Reff', 'X0', 'Xf', 'r0', 'rf'],
      dtype=object)

In [44]:
gammaDF = get_params(ParamEstimatesDf,'abbreviation','gamma','y') 

Available countries are ['China']
Enter country 


 Non per reviewed values will be used 


In [45]:
gammaDF

,peer_review,name,abbreviation,units,country,start_date,end_date,value_type,value,uncertainty_type,lower_bound,upper_bound,population,age_range,method_description,data_description,title_publication,type_publication,authors,publication_URL


## Alpha - reporting rate

In [69]:
AlphaDF = get_params(ParamEstimatesDf,'abbreviation','alpha','y') 

Available countries are ['China']
Enter country China


 Peer reviewed values are available 
peer_review: positive
name: reporting rate
abbreviation: alpha
units: percent
country: China
start_date: 2020-01-24
end_date: 2020-02-03
value_type: median
value: 75.00%
uncertainty_type: confidence interval 95%
lower_bound: 60.00%
upper_bound: 69.00%
population: general population
age_range: Unspecified
method_description: re-estimated system parameters using metapopulation model-inference framework
and city-level daily cases reported between jan 24 and feb 8
data_description: Daily numbers of travelers between 375 Chinese cities during the Spring Festival
period (Chunyun) were derived from human mobility data collected by the Tencent 
Location-based Service (LBS) during the 2018 Chunyun period (February 1 to March 12, 2018)
title_publication: Substantial undocumented infection facilitates the rapid dissemination of novel coronavirus (COVID-19)
type_publication: journal article
authors: Li R.,

In [70]:
AlphaDF

,peer_review,name,abbreviation,units,country,start_date,end_date,value_type,value,uncertainty_type,lower_bound,upper_bound,population,age_range,method_description,data_description,title_publication,type_publication,authors,publication_URL
183,positive,reporting rate,alpha,percent,China,2020-01-24,2020-02-03,median,75.00%,confidence interval 95%,60.00%,69.00%,general population,Unspecified,re-estimated system parameters using metapopul...,Daily numbers of travelers between 375 Chinese...,Substantial undocumented infection facilitates...,journal article,"Li R., Pei S., Chen B., Song Y., Zhang T., Yan...",https://science.sciencemag.org/content/early/2...


In [71]:
alpha = float(AlphaDF.value.values[0].split('.')[0])/100

In [72]:
alpha


0.75

## Beta

In [73]:
BetaDf = get_params(ParamEstimatesDf,'abbreviation','beta','y') 

Available countries are ['China' 'Brazil']
Enter country China


 Peer reviewed values are available 
peer_review: positive
name: transmission rate
abbreviation: beta
units: nan
country: China
start_date: 2020-01-10
end_date: 2020-01-23
value_type: median
value: 1.12
uncertainty_type: confidence interval 95%
lower_bound: 1.04
upper_bound: 1.18
population: general population
age_range: Unspecified
method_description: developed a mathematical model that simulates the spatio-temporal 
dynamics of infections among 375 Chinese cities
data_description: Daily numbers of travelers between 375 Chinese cities during the Spring Festival
period (Chunyun) were derived from human mobility data collected by the Tencent 
Location-based Service (LBS) during the 2018 Chunyun period (February 1 to March 12, 2018)
title_publication: Substantial undocumented infection facilitates the rapid dissemination of novel coronavirus (COVID-19)
type_publication: journal article
authors: Li R., Pei S., Chen B., Song 

In [74]:
BetaDf

,peer_review,name,abbreviation,units,country,start_date,end_date,value_type,value,uncertainty_type,lower_bound,upper_bound,population,age_range,method_description,data_description,title_publication,type_publication,authors,publication_URL
178,positive,transmission rate,beta,NaN,China,2020-01-10,2020-01-23,median,1.12,confidence interval 95%,1.04,1.18,general population,Unspecified,developed a mathematical model that simulates ...,Daily numbers of travelers between 375 Chinese...,Substantial undocumented infection facilitates...,journal article,"Li R., Pei S., Chen B., Song Y., Zhang T., Yan...",https://science.sciencemag.org/content/early/2...
184,positive,transmission rate,beta,NaN,China,2020-01-24,2020-02-03,median,0.52,confidence interval 95%,0.39,0.71,general population,Unspecified,re-estimated system parameters using metapopul...,Daily numbers of travelers between 375 Chinese...,Substantial undocumented infection facilitates...,journal article,"Li R., Pei S., Chen B., Song Y., Zhang T., Yan...",https://science.sciencemag.org/content/early/2...
187,positive,transmission rate,beta,NaN,China,2020-01-24,2020-02-08,median,0.35,confidence interval 95%,0.27,0.5,general population,Unspecified,re-estimated system parameters using metapopul...,Daily numbers of travelers between 375 Chinese...,Substantial undocumented infection facilitates...,journal article,"Li R., Pei S., Chen B., Song Y., Zhang T., Yan...",https://science.sciencemag.org/content/early/2...


In [75]:
BetaDf.value.astype(float).mean()

0.6633333333333334

## Beta

In [76]:
BetaDf = get_params(ParamEstimatesDf,'name','transmission rate','y') 

South Africa value available 


 Non per reviewed values will be used 
peer_review: Not_Available
name: transmission rate
abbreviation: X0
units: day^-1
country: South Africa
start_date: 2019-12-31
end_date: Unspecified
value_type: mean
value: 0.15
uncertainty_type: Unspecified
lower_bound: Unspecified
upper_bound: Unspecified
population: general population
age_range: Unspecified
method_description: Model spread of COVID-19 as partially observed Markov process with real-valued states S, I, and R and integer-valued states C, Y, D, and Y to model sampling into the data; Second model allows for time variable detection probability
data_description: Data from JHU GitHub repository for case count and deaths to 16 April
title_publication: DECLINE IN GLOBAL TRANSMISSION RATES OF COVID-19
type_publication: preprint
authors: Romero-Severson E., Hengartner N., Meadors G., Ke R.
publication_URL: https://www.medrxiv.org/content/10.1101/2020.04.18.20070771v2


peer_review: Not_Available
name: transm

In [77]:
BetaDf

,peer_review,name,abbreviation,units,country,start_date,end_date,value_type,value,uncertainty_type,lower_bound,upper_bound,population,age_range,method_description,data_description,title_publication,type_publication,authors,publication_URL
706,Not_Available,transmission rate,X0,day^-1,South Africa,2019-12-31,Unspecified,mean,0.15,Unspecified,Unspecified,Unspecified,general population,Unspecified,Model spread of COVID-19 as partially observed...,Data from JHU GitHub repository for case count...,DECLINE IN GLOBAL TRANSMISSION RATES OF COVID-19,preprint,"Romero-Severson E., Hengartner N., Meadors G.,...",https://www.medrxiv.org/content/10.1101/2020.0...
707,Not_Available,transmission rate,Xf,day^-1,South Africa,Unspecified,2020-04-16,mean,0.15,Unspecified,Unspecified,Unspecified,general population,Unspecified,Model spread of COVID-19 as partially observed...,Data from JHU GitHub repository for case count...,DECLINE IN GLOBAL TRANSMISSION RATES OF COVID-19,preprint,"Romero-Severson E., Hengartner N., Meadors G.,...",https://www.medrxiv.org/content/10.1101/2020.0...


In [78]:
BetaDf.value.astype(float).mean()

0.15

In [79]:
ValidAbreviation

array([nan, 'R0', 'k', 'CFR', 'RE', 'gamma', 'r', 'R', 'Rt', 'K', 'Rv',
       'Z', 'beta', 'alpha', 'IFR', 'Reff', 'X0', 'Xf', 'r0', 'rf'],
      dtype=object)

In [95]:
t_infDF = get_params(ParamEstimatesDf,'name','time from symptom onset to recovery','y') 

Available countries are ['China']
Enter country China


 Non per reviewed values will be used 
peer_review: Not_Available
name: time from symptom onset to recovery
abbreviation: nan
units: days
country: China
start_date: 2020-01-14
end_date: 2020-02-12
value_type: median
value: 20.3
uncertainty_type: 95% confidence interval
lower_bound: 19.4
upper_bound: 21.3
population: 228 cases with known outcomes
age_range: Unspecified
method_description: time from symptom onset to recovery estimated using lognormal parametric survival methods
data_description: based on 228 cases with known outcomes
title_publication: Epidemiology and Transmission of COVID-19 in Shenzhen China: Analysis of 391 cases and 1,286 of their close contacts
type_publication: preprint
authors: Qifang Bi,Yongsheng Wu, Shujiang Mei, Chenfei Ye, Xuan Zou, Zhen Zhang, Xiaojian Liu, Lan Wei, Shaun A. Truelove
Tong Zhang, Wei Gao, Cong Cheng, Xiujuan Tang, Xiaoliang Wu, Yu Wu, Binbin Sun, Suli Huang, Yu Sun, Juncen Zhang, Ting Ma

In [96]:
t_infDF

,peer_review,name,abbreviation,units,country,start_date,end_date,value_type,value,uncertainty_type,lower_bound,upper_bound,population,age_range,method_description,data_description,title_publication,type_publication,authors,publication_URL
144,Not_Available,time from symptom onset to recovery,NaN,days,China,2020-01-14,2020-02-12,median,20.3,95% confidence interval,19.4,21.3,228 cases with known outcomes,Unspecified,time from symptom onset to recovery estimated ...,based on 228 cases with known outcomes,Epidemiology and Transmission of COVID-19 in S...,preprint,"Qifang Bi,Yongsheng Wu, Shujiang Mei, Chenfei ...",https://www.medrxiv.org/content/10.1101/2020.0...
145,Not_Available,time from symptom onset to recovery,NaN,days,China,2020-01-14,2020-02-12,median,21.2,95% confidence interval,20.2,22.3,228 cases with known outcomes,0-9,time from symptom onset to recovery estimated ...,based on 228 cases with known outcomes,Epidemiology and Transmission of COVID-19 in S...,preprint,"Qifang Bi,Yongsheng Wu, Shujiang Mei, Chenfei ...",https://www.medrxiv.org/content/10.1101/2020.0...
146,Not_Available,time from symptom onset to recovery,NaN,days,China,2020-01-14,2020-02-12,median,17.5,95% confidence interval,15.3,20,228 cases with known outcomes,10-19,time from symptom onset to recovery estimated ...,based on 228 cases with known outcomes,Epidemiology and Transmission of COVID-19 in S...,preprint,"Qifang Bi,Yongsheng Wu, Shujiang Mei, Chenfei ...",https://www.medrxiv.org/content/10.1101/2020.0...
147,Not_Available,time from symptom onset to recovery,NaN,days,China,2020-01-14,2020-02-12,median,19.1,95% confidence interval,15.8,22.9,228 cases with known outcomes,20-29,time from symptom onset to recovery estimated ...,based on 228 cases with known outcomes,Epidemiology and Transmission of COVID-19 in S...,preprint,"Qifang Bi,Yongsheng Wu, Shujiang Mei, Chenfei ...",https://www.medrxiv.org/content/10.1101/2020.0...
148,Not_Available,time from symptom onset to recovery,NaN,days,China,2020-01-14,2020-02-12,median,19.2,95% confidence interval,17.5,21,228 cases with known outcomes,30-39,time from symptom onset to recovery estimated ...,based on 228 cases with known outcomes,Epidemiology and Transmission of COVID-19 in S...,preprint,"Qifang Bi,Yongsheng Wu, Shujiang Mei, Chenfei ...",https://www.medrxiv.org/content/10.1101/2020.0...
149,Not_Available,time from symptom onset to recovery,NaN,days,China,2020-01-14,2020-02-12,median,19.2,95% confidence interval,18,20.5,228 cases with known outcomes,40-49,time from symptom onset to recovery estimated ...,based on 228 cases with known outcomes,Epidemiology and Transmission of COVID-19 in S...,preprint,"Qifang Bi,Yongsheng Wu, Shujiang Mei, Chenfei ...",https://www.medrxiv.org/content/10.1101/2020.0...
150,Not_Available,time from symptom onset to recovery,NaN,days,China,2020-01-14,2020-02-12,median,21.6,95% confidence interval,20,23.4,228 cases with known outcomes,50-59,time from symptom onset to recovery estimated ...,based on 228 cases with known outcomes,Epidemiology and Transmission of COVID-19 in S...,preprint,"Qifang Bi,Yongsheng Wu, Shujiang Mei, Chenfei ...",https://www.medrxiv.org/content/10.1101/2020.0...
151,Not_Available,time from symptom onset to recovery,NaN,days,China,2020-01-14,2020-02-12,median,22.4,95% confidence interval,20.8,24.1,228 cases with known outcomes,60-69,time from symptom onset to recovery estimated ...,based on 228 cases with known outcomes,Epidemiology and Transmission of COVID-19 in S...,preprint,"Qifang Bi,Yongsheng Wu, Shujiang Mei, Chenfei ...",https://www.medrxiv.org/content/10.1101/2020.0...
152,Not_Available,time from symptom onset to recovery,NaN,days,China,2020-01-14,2020-02-12,median,22.9,95% confidence interval,21.2,24.7,228 cases with known outcomes,70-130,time from symptom onset to recovery estimated ...,based on 228 cases with known outcomes,Epidemiology and Transmission of COVID-19 in S...,preprint,"Qifang Bi,Yongsheng Wu, Shujiang Mei, Chenfei ...",https:/

In [98]:
t_infDF.value.astype(float).median()

20.75

## Check reff

In [ ]:
ValidName

In [ ]:
ParamEstimatesDf[(ParamEstimatesDf['abbreviation'] == 'IFR')][ColToList]
#IFRDf = get_params(ParamEstimatesDf,'abbreviation','IFR','n') 

In [ ]:
ParamEstimatesDf['peer_review'].unique()

# This is working for peer_reviewed params

In [52]:
DfToUse = ParamEstimatesDf[ParamEstimatesDf['peer_review'] == 'positive']

In [53]:
MyResult = {}
for Names in DfToUse['name'].unique():
    print(f"Doing parameter {Names}")
    MyResult[Names] = (get_params(ParamEstimatesDf,'name', Names,'n'))
    #break
    

Doing parameter basic reproduction number
Enter country China


 Peer reviewed values are available 
peer_review: positive
name: basic reproduction number
abbreviation: R0
units: Nb
country: China
start_date: 2020-01-09
end_date: 2020-01-11
value_type: Unspecified
value: 0.3
uncertainty_type: confidence interval 95%
lower_bound: 0.17
upper_bound: 0.44
population: general population
age_range: Unspecified
method_description: Using the method described by Cauchemez, assuming that the market was the only source of zoonotic infections
data_description: first 41 confirmed cases
title_publication: Real-time tentative assessment of the epidemiological characteristics of novel coronavirus infections in Wuhan, China, as at 22 January 2020
type_publication: report
authors: Wu P., Hao X., Lau EHY., Wong JY., Leung KSM., Wu JT., Cowling BJ., Leung GB.  
publication_URL: https://eurosurveillance.org/content/10.2807/1560-7917.ES.2020.25.3.2000044#r16


peer_review: positive
name: basic reproduction 

Enter country China


 Peer reviewed values are available 
peer_review: positive
name: incubation period
abbreviation: nan
units: days
country: China
start_date: 2019-12-01
end_date: 2019-02-04
value_type: median
value: 5.2
uncertainty_type: confidence interval 95%
lower_bound: 4.4
upper_bound: 6
population: general population
age_range: Unspecified
method_description: estimated incubation time using parametric accelerated time failure model
data_description: based on confirmed, publicly reported cases outside of Wuhan
title_publication: The incubation period of 2019-nCoV from publicly reported confirmed cases: estimation and application
type_publication: journal article
authors: Lauer SA., Grantz KH., Bi Q., Jones FK., Zheng Q., Meredith H., Azman AS., Reich NG., Lessler J.
publication_URL: https://annals.org/aim/fullarticle/2762808/incubation-period-coronavirus-disease-2019-covid-19-from-publicly-reported


peer_review: positive
name: incubation period
abbreviation: nan
units: days
c

Enter country China


 Peer reviewed values are available 
peer_review: positive
name: effective reproduction number
abbreviation: Re
units: Nb
country: China
start_date: 2020-01-10
end_date: 2020-01-23
value_type: median
value: 2.38
uncertainty_type: confidence interval 95%
lower_bound: 2.04
upper_bound: 2.77
population: general population
age_range: Unspecified
method_description: developed a mathematical model that simulates the spatio-temporal 
dynamics of infections among 375 Chinese cities
data_description: Daily numbers of travelers between 375 Chinese cities during the Spring Festival
period (Chunyun) were derived from human mobility data collected by the Tencent 
Location-based Service (LBS) during the 2018 Chunyun period (February 1 to March 12, 2018)
title_publication: Substantial undocumented infection facilitates the rapid dissemination of novel coronavirus (COVID-19)
type_publication: journal article
authors: Li R., Pei S., Chen B., Song Y., Zhang T., Yang W., Shaman J.
p

Enter country China


 Peer reviewed values are available 
peer_review: positive
name: latent period
abbreviation: Z
units: days
country: China
start_date: 2020-01-10
end_date: 2020-01-23
value_type: median
value: 3.69
uncertainty_type: confidence interval 95%
lower_bound: 3.28
upper_bound: 4.03
population: general population
age_range: Unspecified
method_description: developed a mathematical model that simulates the spatio-temporal 
dynamics of infections among 375 Chinese cities
data_description: Daily numbers of travelers between 375 Chinese cities during the Spring Festival
period (Chunyun) were derived from human mobility data collected by the Tencent 
Location-based Service (LBS) during the 2018 Chunyun period (February 1 to March 12, 2018)
title_publication: Substantial undocumented infection facilitates the rapid dissemination of novel coronavirus (COVID-19)
type_publication: journal article
authors: Li R., Pei S., Chen B., Song Y., Zhang T., Yang W., Shaman J.
publication_URL:

Enter country China


 Peer reviewed values are available 
peer_review: positive
name: proportion of hospitalized cases on mechanical ventilation
abbreviation: nan
units: nan
country: China
start_date: 2020-01-31
end_date: 2020-02-16
value_type: Unspecified
value: 0
uncertainty_type: Unspecified
lower_bound: Unspecified
upper_bound: Unspecified
population: Patients under age 18 with confirmed 2019-nCoV ARD managed at Xian Eight Hospital in Shaanxi, China
age_range: 0-18
method_description: Retrospective reivew of medical records of 25 adults and 7 children with confirmed 2019-nCov ARD
data_description: medical records of 7 coronavirus-infected children
title_publication: Differences in 2019-Coronavirus-infected children and adults
type_publication: Journal Article
authors: Han Y., Feng Z., Sun L., Ren X., Wang H., Xue Y., Wang Y.
publication_URL: https://onlinelibrary-wiley-com.pitt.idm.oclc.org/doi/abs/10.1002/jmv.25835


peer_review: positive
name: proportion of hospitalized cases on

Enter country United States


 Peer reviewed values are available 
peer_review: positive
name: time from hospitalization to mechanical ventilation
abbreviation: nan
units: days
country: United States
start_date: 2020-02-20
end_date: 2020-03-05
value_type: mean
value: 1.5
uncertainty_type: Unspecified
lower_bound: 0
upper_bound: 12
population: Patients with confirmed SARS-CoV-2 infection admitted to ICU
age_range: Unspecified
method_description: single center review of medical records
data_description: medical records of 21 critically ill patients with covid-19
title_publication: Characteristics and Outcomes of 21 Critically Ill Patients With COVID-19 in Washington State
type_publication: Journal Article
authors: Arentz M., Yim E., Klaff L., Lokhandwala S., Riedo FX., Chong M., Lee M
publication_URL: https://jamanetwork-com.pitt.idm.oclc.org/journals/jama/fullarticle/2763485


Doing parameter proportion of icu patients that die
Available countries are ['Italy' 'China']
Enter country Chi

Enter country China


 Peer reviewed values are available 
peer_review: positive
name: proportion of hospitalized cases admitted to icu
abbreviation: nan
units: nan
country: China
start_date: 2020-01-31
end_date: 2020-02-16
value_type: Unspecified
value: 0
uncertainty_type: Unspecified
lower_bound: Unspecified
upper_bound: Unspecified
population: Patients under age 18 with confirmed 2019-nCoV ARD managed at Xian Eight Hospital in Shaanxi, China
age_range: 0-18
method_description: Retrospective reivew of medical records of 25 adults and 7 children with confirmed 2019-nCov ARD
data_description: medical records of 7 coronavirus-infected children
title_publication: Differences in 2019-Coronavirus-infected children and adults
type_publication: Journal Article
authors: Han Y., Feng Z., Sun L., Ren X., Wang H., Xue Y., Wang Y.
publication_URL: https://onlinelibrary-wiley-com.pitt.idm.oclc.org/doi/abs/10.1002/jmv.25835


peer_review: positive
name: proportion of hospitalized cases admitted to 

Enter country China


 Peer reviewed values are available 
peer_review: positive
name: proportion of infections acquired in hospital settings
abbreviation: nan
units: nan
country: China
start_date: 2020-01-01
end_date: 2020-02-03
value_type: Unspecified
value: 0.413043
uncertainty_type: Unspecified
lower_bound: Unspecified
upper_bound: Unspecified
population: Hospitalized patients with confirmed novel coronavirus infected pneumonia (NCIP) ages 42 to 68
age_range: 42-68
method_description: Single center retrospective review of medical records
data_description: medical records of 138 patients
title_publication: Clinical Characteristics of 138 Hospitalized Patients With 2019 Novel Coronavirus–Infected Pneumonia in Wuhan, China
type_publication: Journal Article
authors: Wang D., Hu B., Hu C., Zhu F., Liu X., Zhang J., Wang B., Xiang H., Cheng Z., Xiong Y., Zhao Y., Li Y., Wang X., Peng Z.
publication_URL: https://jamanetwork-com.pitt.idm.oclc.org/journals/jama/fullarticle/2761044


Doing p

Enter country China


 Peer reviewed values are available 
peer_review: positive
name: doubling time
abbreviation: nan
units: days
country: China
start_date: 2020-01-15
end_date: 2020-01-20
value_type: Unspecified
value: 2.7
uncertainty_type: confidence interval 95%
lower_bound: 2.3
upper_bound: 3.3
population: nan
age_range: Unspecified
method_description: computed likelihood of arrival times of first known cases in provinces outside Hubei as function of exponential growing population of infected persons in Wuhan before late January
data_description: Used publicly available reports of 140 confirmed cases from 15 Jan to 20 Jan from China CDC & Baidu migration server to estimate number of daily travelers in and out of Wuhan
title_publication: High Contagiousness and Rapid Spread of Severe Acute Respiratory Syndrome Coronavirus 2
type_publication: journal article
authors: Sanche S., Lin TY., Xu C., Romero-Severson E., Hengartner N., Ke R.
publication_URL: https://wwwnc.cdc.gov/eid/articl

Enter country China


 Peer reviewed values are available 
peer_review: positive
name: serial interval
abbreviation: nan
units: days
country: China
start_date: 2020-12-24
end_date: 2020-02-17
value_type: mean
value: 5.1
uncertainty_type: confidence interval 95%
lower_bound: 1.3
upper_bound: 11.6
population: 35 secondary cases and 28 corresponding primary cases
age_range: Unspecified
method_description: Serial interval estimated by fitting a gamma distribution to the lag between the dates of symptom onset in the index and secondary cases
data_description: Individual records of lab-confirmed COVID-19 cases from websites of national, provicinal, and municipal health commissions and websites of national and local government news media; Daily aggregated data of number of cumulative cases in mainland China from official websites of national, provincial, and municipal health commissions
title_publication: Evolving epidemiology and transmission dynamics of coronavirus disease 2019 outside Hube

In [54]:
#
# This will now summarise the values into another dataframe
#
TMP = []
for key in MyResult.keys():
    try:
        MyResult[key] = MyResult[key][~MyResult[key]['lower_bound'].str.contains("Unspecified", na=False)]
        MyResult[key] = MyResult[key][~MyResult[key]['upper_bound'].str.contains("Unspecified", na=False)]
        TMP.append((MyResult[key]['abbreviation'].unique()[0],
                    key,MyResult[key]['country'].unique()[0],
                    (MyResult[key]['lower_bound'].astype(float).min().round(2)),
                    MyResult[key]['upper_bound'].astype(float).max().round(2),
                    MyResult[key]['value'].astype(float).median().round(2),
                    MyResult[key]['publication_URL'].unique()[0]
                   )
                  )
        print(
            f"{MyResult[key]['abbreviation'].unique()[0]} {key}\
            {MyResult[key]['country'].unique()[0]}\
            {MyResult[key]['lower_bound'].astype(float).min():.2f}\
            {MyResult[key]['upper_bound'].astype(float).max():.2f}\
            {MyResult[key]['value'].astype(float).median():.2f}\
            {MyResult[key]['publication_URL'].unique()[0]}"
            )
        
    except:
        MyResult[key] = MyResult[key][~MyResult[key]['lower_bound'].str.contains("Unspecified", na=False)]
        MyResult[key] = MyResult[key][~MyResult[key]['upper_bound'].str.contains("Unspecified", na=False)]
        print(f"{key}': '{MyResult[key]['value']}")
        #TMP.append((MyResult[key]['country'],MyResult[key]['lower_bound'],
        #            MyResult[key]['upper_bound'],MyResult[key]['value'])
        #           )

R0 basic reproduction number            China            0.17            8.90            2.24            https://eurosurveillance.org/content/10.2807/1560-7917.ES.2020.25.3.2000044#r16
nan proportion of hospitalized cases that die            China            0.04            0.32            0.14            https://eurosurveillance.org/content/10.2807/1560-7917.ES.2020.25.3.2000044
nan incubation period            China            1.80            12.40            5.20            https://annals.org/aim/fullarticle/2762808/incubation-period-coronavirus-disease-2019-covid-19-from-publicly-reported
nan ascertainment rate            China            0.00            2.00            0.20            http://medrxiv.org/content/early/2020/02/09/2020.02.07.20021071.abstract
nan cumulative case count            China            2974.00            130330.00            16829.00            https://www.thelancet.com/journals/lancet/article/PIIS0140-6736(20)30260-9/fulltext
Re effective reproduction numb

In [55]:
TMPDF = pd.DataFrame(data = TMP, columns=["Param","ParamName","Country","LowerBound","UpperBound","Value","URL"])

In [56]:
TMPDF

,Param,ParamName,Country,LowerBound,UpperBound,Value,URL
0,R0,basic reproduction number,China,0.17,8.90,2.24,https://eurosurveillance.org/content/10.2807/1...
1,NaN,proportion of hospitalized cases that die,China,0.04,0.32,0.14,https://eurosurveillance.org/content/10.2807/1...
2,NaN,incubation period,China,1.80,12.40,5.20,https://annals.org/aim/fullarticle/2762808/inc...
3,NaN,ascertainment rate,China,0.00,2.00,0.20,http://medrxiv.org/content/early/2020/02/09/20...
4,NaN,cumulative case count,China,2974.00,130330.00,16829.00,https://www.thelancet.com/journals/lancet/arti...
5,Re,effective reproduction number,China,0.76,2.77,1.36,https://science.sciencemag.org/content/early/2...
6,Z,latent period,China,3.26,4.06,3.60,https://science.sciencemag.org/content/early/2...
7,beta,transmission rate,China,0.27,1.18,0.52,https://science.sciencemag.org/content/early/2...
8,NaN,time from symptom onset to hospitalization,China,0.00,14.00,4.70,https://onlinelibrary-wiley-com.pitt.idm.oclc....
9,NaN,time from hospitalization to mechanical ventil...,United States,0.00,12.00,1.50,https://jamanetwork-com.pitt.idm.oclc.org/jour...


In [63]:
# Doing for params by abbreviation - this is for cross-checking

In [64]:
ToCrossCheck = False

In [66]:
if ToCrossCheck:
    MyResult = {}
    for Names in DfToUse['abbreviation'].unique():
        print(f"Doing parameter {Names}")
        MyResult[Names] = (get_params(ParamEstimatesDf,'abbreviation', Names,'n'))
    #
    # This will now summarise the values into another dataframe
    #
    TMP = []
    for key in MyResult.keys():
        try:
            MyResult[key] = MyResult[key][~MyResult[key]['lower_bound'].str.contains("Unspecified", na=False)]
            MyResult[key] = MyResult[key][~MyResult[key]['upper_bound'].str.contains("Unspecified", na=False)]
            TMP.append((MyResult[key]['abbreviation'].unique()[0],
                        key,MyResult[key]['country'].unique()[0],
                        (MyResult[key]['lower_bound'].astype(float).min().round(2)),
                        MyResult[key]['upper_bound'].astype(float).max().round(2),
                        MyResult[key]['value'].astype(float).median().round(2),
                        MyResult[key]['publication_URL'].unique()[0]
                       )
                      )
            print(
                f"{MyResult[key]['abbreviation'].unique()[0]} {key}\
                {MyResult[key]['country'].unique()[0]}\
                {MyResult[key]['lower_bound'].astype(float).min():.2f}\
                {MyResult[key]['upper_bound'].astype(float).max():.2f}\
                {MyResult[key]['value'].astype(float).median():.2f}\
                {MyResult[key]['publication_URL'].unique()[0]}"
                )

        except:
            MyResult[key] = MyResult[key][~MyResult[key]['lower_bound'].str.contains("Unspecified", na=False)]
            MyResult[key] = MyResult[key][~MyResult[key]['upper_bound'].str.contains("Unspecified", na=False)]
            print(f"{key}': '{MyResult[key]['value']}")
            #TMP.append((MyResult[key]['country'],MyResult[key]['lower_bound'],
            #            MyResult[key]['upper_bound'],MyResult[key]['value'])
            #           )

In [67]:
## Getting parms fro South Africa

In [116]:
#MySAResult = {}
#for Names in DfToUse['name'].unique():
#    print(f"Doing parameter {Names}")
#    MySAResult[Names] = (get_params(ParamEstimatesDf,'name', Names,'y'))
    #break

In [117]:
Midas_SAmodel ={
    "R_0" : R0_SA,
    "R_t" : TMPDF[TMPDF['Param'] == 'Re']['Value'].values[0], # peer reviewed value
    "t_inc" : TMPDF[TMPDF['ParamName'] == 'incubation period']['Value'].values[0], # peer reviewed value
    "t_inf" : t_infDF.value.astype(float).median(), # non-peer reviewed values
    "t_hosp" : TMPDF[TMPDF['ParamName'] == 'duration of hospitalization']['Value'].values[0],
    "t_crit" : TMPDF[TMPDF['ParamName'] == 'time from hospitalization to death']['Value'].values[0],
    "m_a" : 0.8,
    "c_a" : 0.1,
    "f_a" : 0.3,
    "t_icu" : TMPDF[TMPDF['ParamName'] == 'duration of icu stay']['Value'].values[0]
}


In [118]:
Midas_SAmodel

{'R_0': 2.7,
 'R_t': 1.36,
 't_inc': 5.2,
 't_inf': 20.75,
 't_hosp': 10.0,
 't_crit': 11.2,
 'm_a': 0.8,
 'c_a': 0.1,
 'f_a': 0.3,
 't_icu': 9.0}

In [119]:
Midas_Peermodel ={
    "R_0" : TMPDF[TMPDF['ParamName'] == 'basic reproduction number']['Value'].values[0],
    "R_t" : TMPDF[TMPDF['Param'] == 'Re']['Value'].values[0], # peer reviewed value
    "t_inc" : TMPDF[TMPDF['ParamName'] == 'incubation period']['Value'].values[0], # peer reviewed value
    "t_inf" : t_infDF.value.astype(float).median(), # non-peer reviewed values
    "t_hosp" : TMPDF[TMPDF['ParamName'] == 'duration of hospitalization']['Value'].values[0],
    "t_crit" : TMPDF[TMPDF['ParamName'] == 'time from hospitalization to death']['Value'].values[0],
    "m_a" : 0.8,
    "c_a" : 0.1,
    "f_a" : 0.3,
    "t_icu" : TMPDF[TMPDF['ParamName'] == 'duration of icu stay']['Value'].values[0]
}

In [120]:
Midas_Peermodel

{'R_0': 2.24,
 'R_t': 1.36,
 't_inc': 5.2,
 't_inf': 20.75,
 't_hosp': 10.0,
 't_crit': 11.2,
 'm_a': 0.8,
 'c_a': 0.1,
 'f_a': 0.3,
 't_icu': 9.0}

In [ ]:
# From Imperial model

- R_0 = 2.4 # basic reproductive number
- t_incu = 5.1 # average incubation period
- t_infect = 6.5 # average infectious period
- t_hosp = 4 # time in hospital before transitioning to either critical or recovered
- t_crit = 10 # average time patient is critical before transition to recovered or dead

- ratio_mild = 0.80 # ratio of infections that are mild.
- ratio_unreported = 0.8 # ratio of total infections that are unreported due to lack of symptoms #5-80% https://www.cebm.net/covid-19/covid-19-what-proportion-are-asymptomatic/
- ratio_asymptomatic = 0.96 # ratio of infections that are either mild or unreported, and may be symptomless (it's a bad variable name!)
- ratio_severe = 1 - ratio_mild
- ratio_crit = 0.20 # ratio of severe cases that become critical
- ratio_fatal = 0.40 # ratio of critical cases that result in death

In [113]:
Imperial_model ={
        # use parameters from the Imperial paper (https://www.imperial.ac.uk/media/imperial-college/medicine/sph/ide/gida-fellowships/Imperial-College-COVID19-NPI-modelling-16-03-2020.pdf)
        "R_0" : 2.4,
        "R_t" : 1.16,#0.7, # since there is no peer reviewed R_t
        "t_inc" : 5.1,
        "t_inf" : 6.5,
        "t_hosp" : 4,
        "t_crit" : 14,
        "m_a" : 0.8,
        "c_a" : 0.2,
        "f_a" : 0.4
}

In [114]:
Imperial_model

{'R_0': 2.4,
 'R_t': 1.16,
 't_inc': 5.1,
 't_inf': 6.5,
 't_hosp': 4,
 't_crit': 14,
 'm_a': 0.8,
 'c_a': 0.2,
 'f_a': 0.4}

In [115]:
Kaggle_model ={
        "R_0" : 3.6,
        "R_t" : 0.7,
        "t_inc" : 5.6,
        "t_inf" : 2.9,
        "t_hosp" : 4,
        "t_crit" : 14,
        "m_a" : 0.8,
        "c_a" : 0.1,
        "f_a" : 0.3
}

# Done for now
# ToDo:
## Ideas to make a one file for all SA params